# An Introduction to Neural Networks

Neural networks are one of the hottest areas of machine learning. Neural network models attract the most press because of their relationship to the human brain enabling the anthropomorphism of artificial intelligence further. Neural networks aim to emulate the neurons of the brain with perceptrons linking them in networks.

Humans still maintain the edge in the comparison of human and computer intelligence and a look at the wetware specs of the brain and the hardware specs of computers gives us an idea as to why.

### Human Brain
* Neuron Switching Time:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$0.001\text{s}$ ($1\text{ms}$)
* Number of Neurons:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$10^{10}$
* Connections per Neuron:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$10^4-10^5$
* Scene Recognition Time:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$~0.1\ Second$

### Computer
* Transistor Switching Time:&nbsp;&nbsp;&nbsp;&nbsp;$~333\ picoseonds$ (3GHz Processor)
* Transistors per Chip:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$8 - 20\ billion$
* Connections per Transisor:&nbsp;&nbsp;&nbsp;$~10$
* Scene Recognition Time:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;No general scene recognition progamme yet exists (The _Inception_ model is getting pretty close).

We may see that computer hardware wins the comparison hands down in switching time and is starting to win in terms of number of transistors/neurons but the human brain seems way more powerful as a general purpose computing unit at the moment. The power of this comes through the connections per neuron. It is currently not forseen that computing hardware of the current paradigm will ever overtake the brain in this respect. The connections between neurons (albeit slow, exhchanging information via synapses) are where the brain learns and maintains its computational superiority.

Furthermore the brain manages this power with power consumption on the order of 100 Watts whereas the computers used to train and build the leading neural network models consume far far more energy. The brain manages to achieve massive parallelism through the connections of neurons from one to another and thereby achieves remarkable efficiency and understanding.

## The Neuron and the Perceptron
The software equivalent of the neuron is the perceptron. A single neuron is able to encode a single binary threshold which could be sufficent for a simple binary classification model. The neuron either fires or doesn't. In the case that the incoming stimulus is sufficient to cause the neuron to fire an action potential is sent down the axon and down the dendrites which then passes the stimulus on to the neurons connected to firing neuron. This threshold will differ from neuron to neuron as will the number and paths of the connections between neurons.

![Neuron Diagram](https://upload.wikimedia.org/wikipedia/commons/b/b5/Neuron.svg "Neuron Diagram")